In [1]:
import torch
import numpy as np
import math
import os
import pytorch_lightning as pl
import torch
from hydra.utils import instantiate

In [2]:
from os import environ
environ["CUDA_VISIBLE_DEVICES"] = "4"

from pathlib import Path
from hydra import initialize, compose
from bliss.main import predict, train

environ["BLISS_HOME"] = str("/home/declan/current/bliss")

In [3]:
os.environ["BLISS_HOME"]

'/home/declan/current/bliss'

In [4]:
os.getcwd()

'/home/declan/current/bliss/case_studies/redshift_estimation/notebooks'

In [5]:
with initialize(config_path="../", version_base=None):
    cfg = compose("redshift", {
        "predict.weight_save_path=/home/declan/current/bliss/redshift_output/version_4/checkpoints/best_encoder.ckpt"
        })

In [6]:
predict_cfg = cfg.predict

In [7]:
encoder = instantiate(predict_cfg.encoder)
enc_state_dict = torch.load(predict_cfg.weight_save_path)
if predict_cfg.weight_save_path.endswith(".ckpt"):
    enc_state_dict = enc_state_dict["state_dict"]
encoder.load_state_dict(enc_state_dict)
dataset = instantiate(predict_cfg.dataset)
trainer = instantiate(predict_cfg.trainer)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
results = trainer.predict(encoder, datamodule=dataset)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [4]
2024-03-18 12:50:44.871401: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 12:50:44.871453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 12:50:44.873666: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-18 12:50:44.883674: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
202

Predicting: 0it [00:00, ?it/s]

In [ ]:
results[0]['mode_cat'].to_dict()['redshifts']